### 모듈 준비 및 프로그래밍 언어 랭킹 수집

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://www.tiobe.com/tiobe-index/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [40]:
# 현재 TIOBE INDEX 상위 9개 언어 가져오기
soup.select("#top20 > tbody > tr:nth-child(1) > td:nth-child(5)")[0].text
langauge_Rank = []
for idx in range(1,10):
    langauge_Rank.append(soup.select(f"#top20 > tbody > tr:nth-child({idx}) > td:nth-child(5)")[0].text)

langauge_Rank

['Python',
 'C',
 'Java',
 'C++',
 'C#',
 'Visual Basic',
 'JavaScript',
 'Assembly language',
 'SQL']

In [41]:
# R_studio만 추가해주자
langauge_Rank.append('R')
langauge_Rank

['Python',
 'C',
 'Java',
 'C++',
 'C#',
 'Visual Basic',
 'JavaScript',
 'Assembly language',
 'SQL',
 'R']

---

### 문제1)

- 과제 설명 
    - 첫 두 페이지에 제시한 TIOBE INDEX의 상위 9개 언어에 R(14위)까지해서 총 10개 언어에 대해 출판 책 정보를 수집하세요.
    - 여러분들이 선택할 수 있는 여러 방법이 있겠지만, 대표적으로는
        - 네이버 API : https://developers.naver.com/docs/search/book/
        - 네이버 책 홈페이지<br>
    등을 이용할 수 있을 겁니다. 특히 주의해야할 점으로는
    - IT 관련 서적에서 펄 이라는 언어 같은 경우는 유명한 작가 펄과 혼돈할 수 있으며
    - Python과 파이썬 처럼 어떤 경우는 영어와 한글을 각각 따로 검색 후 합쳐야 할 수도 있습니다.
    - 만약 여러분들이 네이버 API를 사용하는 경우 일일 검색량 제한에 조심하세요.

In [42]:
# 모듈 부르고
import os
import sys
import urllib.request
import json
import re
import xmltodict

# 한글로 파이썬, 자바, 자바스크립트만 추가해주자 ,,
for name in ("파이썬","자바","자바스크립트"):
    langauge_Rank.append(name)
langauge_Rank

['Python',
 'C',
 'Java',
 'C++',
 'C#',
 'Visual Basic',
 'JavaScript',
 'Assembly language',
 'SQL',
 'R',
 '파이썬',
 '자바',
 '자바스크립트']

In [43]:
# 검색해야할 목록들의 총 건수를 먼저 확인한다.
client_id = '_VDnEM61XdQ8i2xAFdpf'
client_secret = '91Il8L8MqU'

langauge_Search_Cnt = []
for langauge in langauge_Rank:
    url = "https://openapi.naver.com/v1/search/book_adv.xml" +\
        "?query=" + urllib.parse.quote(langauge) +\
        "&d_titl=" + urllib.parse.quote(langauge) +\
        "&d_catg=" + str(280)
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        cnt = json.loads(json.dumps(xmltodict.parse(response_body)))['rss']['channel']['total']
        langauge_Search_Cnt.append(int(cnt))
    else:
        print("Error Code:" + rescode)

In [44]:
for idx in range(len(langauge_Search_Cnt)):
    print(langauge_Rank[idx], ': ', langauge_Search_Cnt[idx])

Python :  207
C :  2575
Java :  773
C++ :  1044
C# :  308
Visual Basic :  353
JavaScript :  199
Assembly language :  3
SQL :  471
R :  406
파이썬 :  766
자바 :  881
자바스크립트 :  311


In [7]:
# 네이버 API Start의 지정 가능 수가 최대 1000이기 때문에 1000이 넘어가는 건수들은 1000으로 조정
for idx in range(len(langauge_Search_Cnt)):
    if langauge_Search_Cnt[idx] > 1000:
        langauge_Search_Cnt[idx] = 1000

for idx in range(len(langauge_Search_Cnt)):
    print(langauge_Rank[idx], ': ', langauge_Search_Cnt[idx])

Python :  207
C :  1000
Java :  773
C++ :  1000
C# :  308
Visual Basic :  353
JavaScript :  199
Assembly language :  3
SQL :  471
R :  406
파이썬 :  766
자바 :  881
자바스크립트 :  311


In [8]:
# xml 태그를 파이썬에서 읽기 쉽게 dict로 바꾸는 ~
json.loads(json.dumps(xmltodict.parse(response_body)))['rss']['channel']['total']
pd.DataFrame(json.loads(json.dumps(xmltodict.parse(response_body)))['rss']['channel']['item'])

,title,link,image,author,price,discount,publisher,pubdate,isbn,description
0,Do it! HTML+CSS+<b>자바스크립트</b> 웹 표준의 정석 (한 권으로 ...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/177/...,고경희,30000,27000,이지스퍼블리싱,20210122,1163032212 9791163032212,"HTML5+CSS3 웹 표준의 정석》의 고경희 저자는 ‘HTML, CSS, <b>자..."
1,모던 <b>자바스크립트</b> Deep Dive (<b>자바스크립트</b>의 기본 ...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/167/...,이웅모,45000,40500,위키북스,20200925,1158392230 9791158392239,269개의 그림과 원리를 파헤치는 설명으로 ‘<b>자바스크립트</b>의 기본 개념과...
2,코딩만 따라 해도 웹페이지가 만들어지는 HTML+CSS+<b>자바스크립트</b>,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/214/...,반병현,24000,21600,생능북스,20220215,8970505369 9788970505367,HTML/CSS/<b>자바스크립트</b> 기초부터 웹 페이지 제작까지 코딩 초보자의...
3,<b>자바스크립트</b> 완벽 가이드,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/223/...,데이비드 플래너건,42000,37800,인사이트,20220331,8966263461 9788966263462,25년 넘게 사랑받은 <b>자바스크립트</b> 프로그래밍 베스트셀러!\nES2020...
4,생활코딩! HTML+CSS+<b>자바스크립트</b> (처음 프로그래밍을 시작하는 입...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/223/...,이고잉,28000,25200,위키북스,20220407,115839327X 9791158393274,난생 처음으로 프로그래밍을 시작하는 분들이라면 생활코딩 강의로 시작하세요!이고잉 님...
5,혼자 공부하는 <b>자바스크립트</b>,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/176/...,윤인성,24000,21600,한빛미디어,20210104,1162243678 9791162243671,1:1 과외하듯 배우는 <b>자바스크립트</b> 프로그래밍 자습서 (최신 <b>자바...
6,Do it! <b>자바스크립트</b> 입문,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/212/...,고경희,18000,16200,이지스퍼블리싱,20211101,1163033081 9791163033080,"프런트엔드, 백엔드에 모두 쓰는 <b>자바스크립트</b> 실무 문법\n웹 도서 1위..."
7,구글 브레인 팀에게 배우는 딥러닝 with TensorFlow.js (<b>자바스크...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/223/...,샨칭 차이|스탠 바일시|에릭 닐슨|프랑소와 숄레,44000,39600,길벗,20220331,1165219239 9791165219239,이제 <b>자바스크립트</b> 개발자는 파이썬이나 R에 의존하지 않고 TensorF...
8,코어 <b>자바스크립트</b> (핵심 개념과 동작 원리로 이해하는 <b>자바스크립트...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/154/...,정재남,22000,19800,위키북스,20190910,1158391722 9791158391720,<b>자바스크립트</b>의 근간을 이루는 핵심 이론들을 정확하게 이해하는 것을 목표...
9,"실시간 모니터링 시스템을 만들며 정복하는 MEVN (MongoDB, Express,...",http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/211/...,주홍철,32000,28800,비제이퍼블릭,20211018,1165920875 9791165920876,"D3.JS 7.X, MONGODB 5.X, VUE.JS 3.X 반영된 최신 코드로실..."


In [9]:
# 데이터 수집
from tqdm import tqdm_notebook

client_id = '_VDnEM61XdQ8i2xAFdpf'
client_secret = '91Il8L8MqU'

idx = 0
books = pd.DataFrame(columns=['title','link','image','author','price','discount','publisher','pubdate','isbn','description','language'])

for lang_idx in tqdm_notebook(range(len(langauge_Rank))):
    start = 1
    end = langauge_Search_Cnt[lang_idx]
    display = 20

    for start_idx in range(start,end,display):
        # 상세검색 이용, 타이틀에 프로그래밍 언어가 들어가고, 카테고리는 컴퓨터/IT로 한정
        url = "https://openapi.naver.com/v1/search/book_adv.xml" +\
            "?query=" + urllib.parse.quote(langauge_Rank[lang_idx]) +\
            "&d_titl=" + urllib.parse.quote(langauge_Rank[lang_idx]) +\
            "&d_catg=" + str(280) +\
            "&start=" + str(start_idx) +\
            "&display=" + str(display)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            items = json.loads(json.dumps(xmltodict.parse(response_body)))['rss']['channel']['item']
            remove_tag = re.compile('<.*?>')

            for item_idx in range(0,len(items)):
                try:
                    items[item_idx]['title'] = re.sub(remove_tag, '',items[item_idx]['title'])
                    items[item_idx]['description'] = re.sub(remove_tag, '', items[item_idx]['description'])
                    items[item_idx]['description'] = items[item_idx]['description'].replace('\n','')
                except:
                    pass
                items[item_idx]['language'] = langauge_Rank[lang_idx]
                books.loc[idx] = items[item_idx]
                idx += 1
        else:
            print("Error Code:" + rescode)
        

  0%|          | 0/13 [00:00<?, ?it/s]

In [10]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6677 entries, 0 to 6676
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        6677 non-null   object
 1   link         6677 non-null   object
 2   image        6275 non-null   object
 3   author       6670 non-null   object
 4   price        6675 non-null   object
 5   discount     2986 non-null   object
 6   publisher    6677 non-null   object
 7   pubdate      6677 non-null   object
 8   isbn         6674 non-null   object
 9   description  6168 non-null   object
 10  language     6677 non-null   object
dtypes: object(11)
memory usage: 626.0+ KB


---

### 문제2)
- 전세계에서 간행되는 도서에 고유번호를 인가하여 문헌정보와 유통의 효율화를 기대하기 위해 부여되는 일종의 책의 ID 역할을 하는 ISBN 넘버가 있습니다. 
- 문제1에서 수집한 데이터를 ISBN 정보로 unique 검사 후 중복 데이터를 정리하세요.

In [11]:
# 전체 6677개에서 중복 데이터가 1000개 가량 있는 것 같다.
len(books['isbn'].unique())

5515

In [12]:
# 날려 버리자
# https://rfriend.tistory.com/266
# 중복값 제거했을 때 Java와 Java Script가 같이 묶여있는건지 Java Script가 아예 날라가 버리는 경우가 있어서 이를 방지하고자 keep = 'last'
books = books.drop_duplicates(['isbn'], keep='last')
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5515 entries, 3 to 6676
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        5515 non-null   object
 1   link         5515 non-null   object
 2   image        5148 non-null   object
 3   author       5510 non-null   object
 4   price        5513 non-null   object
 5   discount     2443 non-null   object
 6   publisher    5515 non-null   object
 7   pubdate      5515 non-null   object
 8   isbn         5514 non-null   object
 9   description  5061 non-null   object
 10  language     5515 non-null   object
dtypes: object(11)
memory usage: 517.0+ KB


In [13]:
# 한글로 되어있던 파이썬, 자바를 영어 랭귀지로 바꿔주자
PList = list(books[books.language == '파이썬'].index)
JList = list(books[books.language == '자바'].index)
JSList = list(books[books.language == '자바스크립트'].index)

for idx in PList:
    books.loc[idx]['language'] = 'Python'

for idx in JList:
    books.loc[idx]['language'] = 'Java'

for idx in JSList:
    books.loc[idx]['language'] = 'JavaScript'

# 영어로 통일됐당
books.language.unique()

array(['Python', 'C', 'Java', 'C++', 'C#', 'Visual Basic', 'JavaScript',
       'Assembly language', 'SQL', 'R'], dtype=object)

---

### 문제3)
- 문제2의 결과에서 각 언어별 출판물의 양으로 순위를 매겨주세요. 
- 이때 적절히 시각화하여 그 양도 확인해주세요

In [14]:
Lang_Sales = books.groupby('language').count().iloc[:,:1].sort_values(by='title', ascending=False)
Lang_Sales.columns = ['Sales_Volume']
Lang_Sales

,Sales_Volume
language,
Java,1047
C++,975
Python,925
C,614
JavaScript,466
SQL,452
R,393
Visual Basic,341
C#,299


In [15]:
import plotly_express as px
px.bar(Lang_Sales, 
       x = Lang_Sales.index, y='Sales_Volume', color='Sales_Volume', color_continuous_scale='Greens',
       title='Programing Language Book Sales Volume')
# 제일 유명한 Java, Python, C 등이 역시 상위에 랭크되었다.

In [16]:
# 연도별 추이도 보기 위해 연도를 추가해주자
Years = [int(date[:4]) for date in list(books.pubdate)]
books['Year'] = Years   

In [28]:
books.head()

,title,link,image,author,price,discount,publisher,pubdate,isbn,description,language,Year
3,OpenCV.Python 머신러닝 딥러닝 프로그래밍,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/223/...,김동근,26000,23400,가메,20220331,8980783116 9788980783113,OPENCV ㆍ PYTHON을 이용한 머신러닝과 딥러닝 프로그래밍을 소개합니다 .-...,Python,2022
4,영상 처리 및 응용 (OpenCV-Python으로 배우는),http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/175/...,정성환,35000,35000,생능출판,20201124,8970504419 9788970504414,『OPENCV-PYTHON으로 배우는 영상 처리 및 응용』 은 〈영상처리란 무엇인가...,Python,2020
5,처음 만나는 AI 수학 with Python (한 권으로 배우는 인공지능 수학 첫걸음),http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/177/...,아즈마 유키나가,23000,20700,영진닷컴,20210115,8931463375 9788931463378,인공지능을 공부하는데 필요한 기초 수학개념을 한 권에 모았다!〈처음 만나는 AI 수...,Python,2021
6,빅데이터분석 및 인공지능 (쉽고 간단한 예제를 이용한 빅데이터분석 및 인공지능 실습...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/180/...,조준모,26000,26000,인피니티북스,20210225,118557879X 9791185578798,이를 위해 현재 각광을 받고 있는 구글의 주피터 노트북(JUPYTER NOTEBOO...,Python,2021
7,Python으로 배우는 문제해결과 인공지능,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/180/...,김현정|오경선,23000,23000,인피니티북스,20210225,1185578811 9791185578811,컴퓨팅 사고는 자신의 영역에 컴퓨터과학 원리와 개념을 활용하여 논리적으로 풀어낼 수...,Python,2021


In [18]:
Lang_Sales_Year = books.groupby(['Year','language']).count().iloc[:,:1]
Lang_Sales_Year.columns = ['Sales_Volume']
# 이상하게 집계된 연도도 있길래 제거
Lang_Sales_Year = Lang_Sales_Year.reset_index().iloc[2:,:]
Lang_Sales_Year.sort_values(by='Year',ascending=True)

px.line(Lang_Sales_Year, x='Year',y='Sales_Volume',color='language')
# 오호랑 파이썬이 2015년부터 쭉쭉 올라오네, R도 같이 올라오다 주춤하고 있구나
# Java가 2000년대 초반에는 1등이었군

---

### 문제4)
- 수집된 정보를 기반으로 우리나라의 컴퓨터 언어 관련 출판사 순위를 확인해 주세요.

In [19]:
publisher_Sales = books.groupby('publisher').count().iloc[:,:1].sort_values(by='title',ascending=False).head(16)
publisher_Sales.columns = ['Sales_Volume']
# Packt(GCOScience)는 우리나라 출판사가 아닌 것
publisher_Sales.drop(['Packt(GCOScience)'], inplace=True)
publisher_Sales

,Sales_Volume
publisher,
한빛미디어,358
에이콘출판,302
정보문화사,243
영진닷컴,216
성안당,122
위키북스,117
길벗,111
인포북,111
삼각형,105


In [20]:
import plotly_express as px
px.bar(publisher_Sales,
       x=publisher_Sales.index, y='Sales_Volume', color='Sales_Volume', color_continuous_scale='Greens',
       title='Programing Language Publisher Sales Volume Top 15')

# 한빛미디어가 1위를 달리고 있다. 
# 15위에 한빛 아카데미도 있는 것으로 보아 한빛 계열사가 우리나라 컴퓨터 언어 출판사를 주름잡고 있는 듯 하다?

In [21]:
# 이번에도 연도별 추세를 살펴보자.

publisher_Sales_Year = books.groupby(['Year','publisher']).count().iloc[2:,:].iloc[:,:1]
publisher_Sales_Year.columns = ['Sales_Volume']
publisher_Sales_Year = publisher_Sales_Year.reset_index()

# top 15만~
publisher_Sales_Year = publisher_Sales_Year[publisher_Sales_Year.publisher.isin(publisher_Sales.index)]
px.line(publisher_Sales_Year, x='Year',y='Sales_Volume',color='publisher')

# 삼각형이라는 곳은 이름을 바꾼걸까 망한걸까
# 최근에는 에이콘 출판이 강세였고 한빛 미디어는 꾸준했던거지, 독보적인 때는 별로 없었네?

---

### 문제5)
- 수집된 정보에서 출판일 기준, 최근 2년간(20년, 21년) 데이터와 그 전 3년(17년, 18년, 19년) 데이터를 비교해주세요 .
- 여기서 비교해야할 대상은 여러분들이 정하세요. 두 기간의 데이터를 관찰해서 특이한 사항들을 정리해주면 됩니다. 
- 이 과정에서 역시 적절한 시각화도 고민해 주세요. 특히 어떤 특정 언어의 변화가 감지되면 강조해주세요.

In [22]:
# 20,21 프로그래밍 언어 도서 언어별 판매량
books20_21 = books[(books['Year'] == 2020) | (books['Year'] == 2021)]
books20_21 = books20_21.groupby('language').count().iloc[:,:1].sort_values(by='title',ascending=False)
books20_21.columns = ['Sales_Volume']

# 17,18,19 프로그래밍 언어 도서 언어별 판매량
books17_19 = books[(books['Year'] == 2017) | (books['Year'] == 2018) | (books['Year'] == 2019)]
books17_19 = books17_19.groupby('language').count().iloc[:,:1].sort_values(by='title',ascending=False)
books17_19.columns = ['Sales_Volume']

In [23]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [24]:
# 일단 판매량부터 비교해볼까?
fig = make_subplots(
        rows=1,cols=2,shared_xaxes=False,
        specs = [[{"type": "bar"}, {"type": "bar"}]],
        subplot_titles = ("17 ~ 19", "20 ~ 21")
)

fig.add_trace(go.Bar(
        x=books17_19.index,
        y=books17_19.Sales_Volume),
        row=1,col=1
)

fig.add_trace(go.Bar(
        x=books20_21.index,
        y=books20_21.Sales_Volume),
        row=1,col=2
)

fig.show()
# 우와 최근 5년은 파이썬이 압도적이다.
# Java와 R은 엎치락 뒤치락

In [25]:
# 근데 파이썬이 전체 기간동안 거의 900권 정도 판매량을 기록했었는데 5년동안 700을 넘네?
# 비율을 확인해보자
(books20_21[books20_21.index=='Python'].Sales_Volume + books17_19[books17_19.index=='Python'].Sales_Volume) / Lang_Sales[Lang_Sales.index == 'Python'].Sales_Volume

# 78% 우와 최근 5년동안 난리가 난거구나

language
Python    0.788108
Name: Sales_Volume, dtype: float64

In [26]:
# 절대적인 판매량 말고, 각 언어 판매량이 해당 기간의 총 판매량에서 얼만큼의 비율을 차지하는지 비교하자.
# 비율 만들어주자
import numpy as np
books17_19['Sales_Rate'] = books17_19['Sales_Volume'] / np.sum(list(books17_19['Sales_Volume'])) * 100
books20_21['Sales_Rate'] = books20_21['Sales_Volume'] / np.sum(list(books20_21['Sales_Volume'])) * 100

fig = make_subplots(
        rows=1,cols=2,shared_xaxes=False,
        specs = [[{"type": "pie"}, {"type": "pie"}]],
        subplot_titles = ("17 ~ 19", "20 ~ 21")
)

fig.add_trace(go.Pie(
        labels = books17_19.index,
        values = books17_19['Sales_Rate'],
        hole=.4),
        row=1,col=1
)

fig.add_trace(go.Pie(
        labels = books20_21.index,
        values = books20_21['Sales_Rate'],
        hole=.4),
        row=1,col=2
)

fig.show()

# 최근 2년은 파이썬이 절반을 차지해버린다. 인기가 하늘을 찌르는구나

In [29]:
# 가격도 살펴보고 싶어서 object type으로 되어있던 걸 int로 바꾸려 했더니 형 변환 오류가 났다.
# 찾아보니까 가격 데이터에 None인 놈들이 두 놈 있네, 일단 없애보자
books = books[books.price.notna()]
books = books.astype({'price':'float'})

# 오예 float으로 바뀌었당
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5513 entries, 3 to 6676
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        5513 non-null   object 
 1   link         5513 non-null   object 
 2   image        5147 non-null   object 
 3   author       5508 non-null   object 
 4   price        5513 non-null   float64
 5   discount     2443 non-null   object 
 6   publisher    5513 non-null   object 
 7   pubdate      5513 non-null   object 
 8   isbn         5512 non-null   object 
 9   description  5060 non-null   object 
 10  language     5513 non-null   object 
 11  Year         5513 non-null   int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 559.9+ KB


In [30]:
# 인기가 많아지면 가격이 비싸질까?를 보고싶었다. 인기의 척도는 판매 건수로 하겠음
# 파이썬을 우선 살펴보자
tmp1 = pd.DataFrame(books[books.language=='Python'].groupby('Year').count().iloc[:,:1]).reset_index() # 판매 건수
tmp2 = pd.DataFrame(books[books.language=='Python'].groupby('Year')['price'].mean().round()).reset_index() # 가격 평균
Sales_Price = pd.merge(tmp1,tmp2)
Sales_Price.columns = ['Year','s_Vol','price']
Sales_Price = Sales_Price.set_index('Year')
Sales_Price

# 어 ,, 시각화해 보기 전 부터 상관이 없다는게 느껴져서 중지! 

,s_Vol,price
Year,,
1998,1,17000.0
2000,2,16300.0
2001,3,19667.0
2002,4,29000.0
2003,1,32000.0
2005,1,35000.0
2007,1,17000.0
2009,1,26000.0
2010,2,25000.0


In [39]:
# 연도별 판매 건수 / 연도별 총 판매 건수
tmp = pd.DataFrame(Lang_Sales_Year.groupby('Year').Sales_Volume.sum())
tmp.columns = ['Total_Volume']
tmp = tmp.reset_index()

each_total_Sales_Volume = pd.merge(Lang_Sales_Year, tmp, how='outer').sort_values(by='Year')
# 비율 생성
each_total_Sales_Volume['Rate'] = each_total_Sales_Volume['Sales_Volume'] / each_total_Sales_Volume['Total_Volume'] * 100
px.line(
    each_total_Sales_Volume, x='Year', y='Rate', color = 'language'
)

# 연도 별로 어떤 프로그래밍 도서가 많이 팔렸는지 비율을 그려봤다.
# 비율에서도 역시 2015년 이후 파이썬이 쭉 치고 올라오는게 보인다.

다만 C언어 검색건수가 2500건이었는데, 이를 온전히 받아와서 분석을 수행했다면, 결과가 조금 달라졌을 수 도 있겠다.<br>
우선은, 전체적인 프로세스를 진행하는 것에 조금 더 중점을 두었었다.

---

### 추가
- 여러분들 수집한 책 목록을 먼저 TIOBE INDEX 20위까지 확장해서 검색해 주세요.
- 검색할때 책 가격과 페이지수도 함께 수집해 주세요.
- 그리고 아래 질문에 답해주세요.
    - 언어별 책의 가격은 어떤 특성이 있나요?
    - 언어별 페이지수도 어떤 특성이 있나요?
    - 페이지 수와 가격 사이에는 어떤 특성이 있나요?

TIOBE INDEX TOP 20

In [46]:
url = "https://www.tiobe.com/tiobe-index/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# 현재 TIOBE INDEX 상위 20개 언어 가져오기
soup.select("#top20 > tbody > tr:nth-child(1) > td:nth-child(5)")[0].text
langauge_Rank = []
for idx in range(1,21):
    langauge_Rank.append(soup.select(f"#top20 > tbody > tr:nth-child({idx}) > td:nth-child(5)")[0].text)

langauge_Rank

['Python',
 'C',
 'Java',
 'C++',
 'C#',
 'Visual Basic',
 'JavaScript',
 'Assembly language',
 'SQL',
 'PHP',
 'R',
 'Delphi/Object Pascal',
 'Go',
 'Swift',
 'Ruby',
 'Classic Visual Basic',
 'Objective-C',
 'Perl',
 'Lua',
 'MATLAB']

In [53]:
# 또 한글도 추가해주고
for name in ("파이썬","자바","자바스크립트","델파이"):
    langauge_Rank.append(name)
langauge_Rank

['Python',
 'C',
 'Java',
 'C++',
 'C#',
 'Visual Basic',
 'JavaScript',
 'Assembly language',
 'SQL',
 'PHP',
 'R',
 'Delphi/Object Pascal',
 'Go',
 'Swift',
 'Ruby',
 'Classic Visual Basic',
 'Objective-C',
 'Perl',
 'Lua',
 'MATLAB',
 '파이썬',
 '자바',
 '자바스크립트',
 '파이썬',
 '자바',
 '자바스크립트',
 '델파이']

In [54]:
langauge_Search_Cnt = []
for langauge in langauge_Rank:
    url = "https://openapi.naver.com/v1/search/book_adv.xml" +\
        "?query=" + urllib.parse.quote(langauge) +\
        "&d_titl=" + urllib.parse.quote(langauge) +\
        "&d_catg=" + str(280)
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        cnt = json.loads(json.dumps(xmltodict.parse(response_body)))['rss']['channel']['total']
        langauge_Search_Cnt.append(int(cnt))
    else:
        print("Error Code:" + rescode)

In [56]:
for idx in range(len(langauge_Search_Cnt)):
    print(langauge_Rank[idx], ': ', langauge_Search_Cnt[idx])

# 일단은 1000개 넘는건 커트(C, C++)
for idx in range(len(langauge_Search_Cnt)):
    if langauge_Search_Cnt[idx] > 1000:
        langauge_Search_Cnt[idx] = 1000

Python :  207
C :  2575
Java :  773
C++ :  1044
C# :  308
Visual Basic :  353
JavaScript :  199
Assembly language :  3
SQL :  471
PHP :  205
R :  406
Delphi/Object Pascal :  0
Go :  66
Swift :  29
Ruby :  11
Classic Visual Basic :  0
Objective-C :  17
Perl :  29
Lua :  5
MATLAB :  187
파이썬 :  766
자바 :  881
자바스크립트 :  311
파이썬 :  766
자바 :  881
자바스크립트 :  311
델파이 :  101


In [57]:
# 데이터 수집
from tqdm import tqdm_notebook

client_id = '_VDnEM61XdQ8i2xAFdpf'
client_secret = '91Il8L8MqU'

idx = 0
books = pd.DataFrame(columns=['title','link','image','author','price','discount','publisher','pubdate','isbn','description','language'])

for lang_idx in tqdm_notebook(range(len(langauge_Rank))):
    start = 1
    end = langauge_Search_Cnt[lang_idx]
    display = 20

    for start_idx in range(start,end,display):
        # 상세검색 이용, 타이틀에 프로그래밍 언어가 들어가고, 카테고리는 컴퓨터/IT로 한정
        url = "https://openapi.naver.com/v1/search/book_adv.xml" +\
            "?query=" + urllib.parse.quote(langauge_Rank[lang_idx]) +\
            "&d_titl=" + urllib.parse.quote(langauge_Rank[lang_idx]) +\
            "&d_catg=" + str(280) +\
            "&start=" + str(start_idx) +\
            "&display=" + str(display)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            items = json.loads(json.dumps(xmltodict.parse(response_body)))['rss']['channel']['item']
            remove_tag = re.compile('<.*?>')

            for item_idx in range(0,len(items)):
                try:
                    items[item_idx]['title'] = re.sub(remove_tag, '',items[item_idx]['title'])
                    items[item_idx]['description'] = re.sub(remove_tag, '', items[item_idx]['description'])
                    items[item_idx]['description'] = items[item_idx]['description'].replace('\n','')
                except:
                    pass
                items[item_idx]['language'] = langauge_Rank[lang_idx]
                books.loc[idx] = items[item_idx]
                idx += 1
        else:
            print("Error Code:" + rescode)
        

  0%|          | 0/27 [00:00<?, ?it/s]

In [59]:
books = books.drop_duplicates(['isbn'], keep='last')
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6084 entries, 3 to 9282
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        6084 non-null   object
 1   link         6084 non-null   object
 2   image        5649 non-null   object
 3   author       6077 non-null   object
 4   price        6082 non-null   object
 5   discount     2618 non-null   object
 6   publisher    6084 non-null   object
 7   pubdate      6084 non-null   object
 8   isbn         6083 non-null   object
 9   description  5538 non-null   object
 10  language     6084 non-null   object
dtypes: object(11)
memory usage: 570.4+ KB


In [60]:
books.language.unique()

array(['Python', 'C', 'Java', 'C++', 'C#', 'Visual Basic', 'JavaScript',
       'Assembly language', 'SQL', 'PHP', 'R', 'Go', 'Swift', 'Ruby',
       'Objective-C', 'Perl', 'Lua', 'MATLAB', '파이썬', '자바', '자바스크립트',
       '델파이'], dtype=object)

In [62]:
# 한글로 되어있던 파이썬, 자바를 영어 랭귀지로 바꿔주자
PList = list(books[books.language == '파이썬'].index)
JList = list(books[books.language == '자바'].index)
JSList = list(books[books.language == '자바스크립트'].index)
DList = list(books[books.language == '델파이'].index)

for idx in PList:
    books.loc[idx]['language'] = 'Python'

for idx in JList:
    books.loc[idx]['language'] = 'Java'

for idx in JSList:
    books.loc[idx]['language'] = 'JavaScript'

for idx in DList:
    books.loc[idx]['language'] = 'Delphi/Object Pascal'

# 영어로 통일됐당
books.language.unique()

array(['Python', 'C', 'Java', 'C++', 'C#', 'Visual Basic', 'JavaScript',
       'Assembly language', 'SQL', 'PHP', 'R', 'Go', 'Swift', 'Ruby',
       'Objective-C', 'Perl', 'Lua', 'MATLAB', 'Delphi/Object Pascal'],
      dtype=object)

In [64]:
# 연도 추가
Years = [int(date[:4]) for date in list(books.pubdate)]
books['Year'] = Years  
books.tail()

,title,link,image,author,price,discount,publisher,pubdate,isbn,description,language,Year
9278,정보처리산업기사 실기 (델파이),http://book.naver.com/bookdb/book_detail.php?b...,None,주병진,10000,None,정보문화사,19990111,8976275284 9788976275288,None,Delphi/Object Pascal,1999
9279,델파이 2 (21일완성),http://book.naver.com/bookdb/book_detail.php?b...,None,DAN OSIER,18000,None,인포북,19970610,8980542712 9788980542710,None,Delphi/Object Pascal,1997
9280,젊은이들과 함께하는 델파이 2,http://book.naver.com/bookdb/book_detail.php?b...,None,안동훈,23000,None,에프원,19961020,898618530X 9788986185300,None,Delphi/Object Pascal,1996
9281,델파이 개발자 가이드,http://book.naver.com/bookdb/book_detail.php?b...,None,KMK,25000,None,삼각형,19951101,8974671468 9788974671464,None,Delphi/Object Pascal,1995
9282,델파이 이렇게 사용한다,http://book.naver.com/bookdb/book_detail.php?b...,None,한민영,13000,None,높이깊이,19950901,8975880281 9788975880285,None,Delphi/Object Pascal,1995


In [72]:
# 페이지 정보 얻는 함수 ~
def get_page_num(soup):
    tmp = soup.find_all(class_ = 'book_info')[0].get_text()

    try:
        result = re.search('페이지\s+\d+', tmp).group()
        return result.split()[1]
    except:
        return np.nan

In [86]:
import time

pages = []
for url in tqdm_notebook(books.link):
    page = requests.get(url)
    try:
        page_num = get_page_num(BeautifulSoup(page.content, 'html.parser'))
        pages.append(page_num)
    except:
        pages.append(np.nan)
# 와 진짜 오래 걸린다

  0%|          | 0/6084 [00:00<?, ?it/s]

In [87]:
len(pages)

6084

In [147]:
# books index 초기화 먼저 ~ 꼭 ~

for idx in range(len(pages)):
    print(pages[idx])
    books.loc[idx,'page'] = pages[idx]

328
620
308
368
304
158
552
480
128
776
560
332
394
400
328
350
506
224
455
690
328
332
366
432
363
208
180
244
nan
nan
489
234
737
215
34
174
475
539
544
320
420
500
459
983
527
310
510
761
455
353
824
368
348
281
203
280
527
390
266
152
120
145
320
444
371
282
nan
334
577
619
416
359
771
456
396
366
624
496
356
79
364
nan
62
149
388
498
426
730
526
441
156
755
483
414
303
800
756
403
236
78
370
492
308
332
471
358
763
371
379
560
326
250
433
352
300
375
162
466
224
227
454
415
172
356
348
384
274
324
327
431
240
308
396
328
450
110
106
442
79
76
542
398
322
166
852
297
366
255
1278
406
170
438
366
308
128
412
122
1311
612
423
420
nan
216
664
776
616
552
624
428
574
576
606
536
380
464
372
780
784
538
1440
408
784
800
656
59
504
641
744
456
591
1199
304
708
664
424
632
596
643
540
516
398
548
474
764
532
600
390
276
240
686
518
528
408
652
668
784
249
462
812
416
321
560
936
368
415
684
404
580
238
1336
544
276
685
578
nan
nan
706
211
406
487
808
404
538
224
186
448
278
264
480
522
66

In [149]:
# 페이지가 잘 들어왔다.
books.head()

,title,link,image,author,price,discount,publisher,pubdate,isbn,description,language,Year,page
0,OpenCV.Python 머신러닝 딥러닝 프로그래밍,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/223/...,김동근,26000,23400,가메,20220331,8980783116 9788980783113,OPENCV ㆍ PYTHON을 이용한 머신러닝과 딥러닝 프로그래밍을 소개합니다 .-...,Python,2022,328
1,영상 처리 및 응용 (OpenCV-Python으로 배우는),http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/175/...,정성환,35000,35000,생능출판,20201124,8970504419 9788970504414,『OPENCV-PYTHON으로 배우는 영상 처리 및 응용』 은 〈영상처리란 무엇인가...,Python,2020,620
2,처음 만나는 AI 수학 with Python (한 권으로 배우는 인공지능 수학 첫걸음),http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/177/...,아즈마 유키나가,23000,20700,영진닷컴,20210115,8931463375 9788931463378,인공지능을 공부하는데 필요한 기초 수학개념을 한 권에 모았다!〈처음 만나는 AI 수...,Python,2021,308
3,빅데이터분석 및 인공지능 (쉽고 간단한 예제를 이용한 빅데이터분석 및 인공지능 실습...,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/180/...,조준모,26000,26000,인피니티북스,20210225,118557879X 9791185578798,이를 위해 현재 각광을 받고 있는 구글의 주피터 노트북(JUPYTER NOTEBOO...,Python,2021,368
4,Python으로 배우는 문제해결과 인공지능,http://book.naver.com/bookdb/book_detail.php?b...,https://bookthumb-phinf.pstatic.net/cover/180/...,김현정|오경선,23000,23000,인피니티북스,20210225,1185578811 9791185578811,컴퓨팅 사고는 자신의 영역에 컴퓨터과학 원리와 개념을 활용하여 논리적으로 풀어낼 수...,Python,2021,304


---

### 언어별 책의 가격은 어떤 특성이 있나요?

In [211]:
books_price = pd.DataFrame(books.groupby('language')['price'].mean().round()).sort_values(by='price',ascending=False)
px.bar(books_price, x=books_price.index, y='price', color = 'price' , color_continuous_scale='Greens', title = 'Price(mean) by Language')

---

### 언어별 페이지수도 어떤 특성이 있나요?

In [209]:
books_page = pd.DataFrame(books.groupby('language')['page'].mean().round()).sort_values(by='page',ascending=False)
px.bar(books_page, x=books_price.index, y='page', color = 'page' , color_continuous_scale='Greens', title = 'Price by Language')

시간 관계상 순위 정도만 알아보았지만, 프로그래밍 언어에 따라 가격이나 페이지에 있어서 패턴이 있을까?<br>
나중에 더 깊게 찾아봐야겠다.

---

### 페이지 수와 가격 사이에는 어떤 특성이 있나요?

In [176]:
# nan값들 처리하고
books = books[books.page.notna()]
books = books[books.price.notna()]

In [184]:
books = books.astype({'price':'float', 'page':'float'})

In [185]:
books = books.sort_values(by=['page','price'])

In [194]:
# 50 page 미만은 버립니다.
books = books[books.page >= 50]

# 가격이 0원인 것도 버립니다.
books = books[books.price > 0]

In [195]:
fig = px.scatter(
    books, x='page', y='price', opacity= 0.65,
    trendline='ols', trendline_color_override='darkblue'
)
fig.show()
# 오 ~ 꽤 연관성이 있어보입니다 ~
# 그런데 R-squared가 낮네용